In [ ]:
import numpy as np
from open3d import *
import torch
import seaborn as sns
import json
import time

In [ ]:
file_name = '5_points_GTv3_05_inst_nostuff.pth'

In [ ]:
view_json = {
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : False,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 23.999622344970703, 29.414335250854492, 12.430385589599609 ],
			"boundingbox_min" : [ -25.999080657958984, -20.583383560180664, -4.4400639533996582 ],
			"field_of_view" : 60.0,
			"front" : [ -0.31103482071161342, 0.90056388039681612, 0.30371374290526221 ],
			"lookat" : [ -1.1835630984909713, 2.2750627512522241, 2.2576575227022864 ],
			"up" : [ -0.030620578232786085, -0.3288943841291509, 0.94387015223334414 ],
			"zoom" : 0.59999999999999987
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

In [ ]:
def calc_color_vec(labels):
    index_label_dict = {int(value):int(index[0]) for index, value in np.ndenumerate(np.unique(labels))}
    color_codes = list(sns.color_palette("husl", len(np.unique(labels))))
    colors = [color_codes[index_label_dict[int(label)]] for label in np.nditer(labels)]
    return colors

In [ ]:
def create_image(pcd, output_path, view_json):
    vis = open3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    view_control = vis.get_view_control()
    view_control.set_front(view_json['trajectory'][0]['front'])
    view_control.set_zoom(view_json['trajectory'][0]['zoom'])
    view_control.set_lookat(view_json['trajectory'][0]['lookat'])
    view_control.set_up(view_json['trajectory'][0]['up'])

    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

    time.sleep(1)
    vis.capture_screen_image(output_path)
    vis.remove_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
    vis.destroy_window()
    del view_control
    del vis

In [ ]:
def load_pcd(path):
    data = torch.load(path)
    cords = data[0]
    colors = data[1]
    colors = (colors + 1) / 2
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(cords)
    pcd.colors = open3d.utility.Vector3dVector(colors)
    return pcd

In [ ]:
def make_path(path):
    return f'{path}/val/{file_name}'

In [ ]:
def make_image_from_path(p):
    file_path = make_path(p)
    print(p)
    pcd = load_pcd(file_path)
    output_path = f'images/{p.split("/")[-1]}.jpg'
    create_image(pcd, output_path, view_json)

In [ ]:
paths = ['data',
         'data\manipulated_data/no_color_',
         'data\manipulated_data/noisy_0.2',
         'data\manipulated_data/noisy_0.5',
         'data\manipulated_data/noisy_0.8',
         'data\manipulated_data/noisy_1.0',
         'data\manipulated_data/sample_farthest_0.6',
         'data\manipulated_data/sample_random_0.85',
         'data\manipulated_data/sample_random_0.6',
         'data\manipulated_data/sample_random_0.5',
         'data\manipulated_data/sample_random_0.3'
]
for path in paths:
    make_image_from_path(path)

## Scence with color

In [ ]:
data = torch.load('data/recorded_data/val/conferenceRoom_2.pth')
cords = data[0]
colors = data[1]
colors = (colors + 1) / 2

In [ ]:
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(cords[mask])
pcd.colors = open3d.utility.Vector3dVector(colors[mask])
open3d.visualization.draw_geometries([pcd])

In [ ]:
vis = open3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)

view_control = vis.get_view_control()
view_control.set_front(view_json['trajectory'][0]['front'])
view_control.set_zoom(view_json['trajectory'][0]['zoom'])
view_control.set_lookat(view_json['trajectory'][0]['lookat'])
view_control.set_up(view_json['trajectory'][0]['up'])
view_control.scale(0.8)

vis.update_geometry(pcd)
vis.poll_events()
vis.update_renderer()

time.sleep(1)
vis.capture_screen_image('testimage.png')
vis.remove_geometry(pcd)
vis.poll_events()
vis.update_renderer()
vis.destroy_window()
del view_control
del vis

## Scence with label

In [ ]:
labels = data[3]

In [ ]:
colors = calc_color_vec(labels)
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(cords)
pcd.colors = open3d.utility.Vector3dVector(colors)
#open3d.visualization.draw_geometries([pcd])